In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pestools as pt
import pyemu
import os, shutil
import re
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
files_to_copy = ['freyberg.hyd','points1.dat.tpl',
                 'freyberg_pp_reg_phimlim26.jcb','freyberg_pp_reg_phimlim26.pst',
                'freyberg_pp_reg_phimlim26.post.cov']
[shutil.copy2(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints',i),i) for i in files_to_copy]



In [ ]:
obslox = pd.read_csv('freyberg.hyd', delim_whitespace=True, usecols = [4,5,6], 
                     index_col=2, skiprows = 1, header=None, names=['X','Y','obsname'])
obslox = obslox.drop([i for i in obslox.index if not i.startswith('cr')], axis=0)
parlox = pd.read_csv('points1.dat.tpl', delim_whitespace=True, usecols=[0,1,2],
                    index_col=0, skiprows=1, header=None, names=['parname','X','Y'])
parlox['pp_num'] = [re.findall('\d+',i)[0] for i in parlox.index.values]

# `pestools` plotting identifiability

In [ ]:
# note that both the Jacobian and the PST file need to be present for this to work
idpar = pt.IdentPar('freyberg_pp_reg_phimlim26.jcb')


In [ ]:
ax=idpar.plot_singular_spectrum()


In [ ]:
idpar.plot_bar(nsingular=12,nbars=50);

### How does this compare with CSS?

In [ ]:
la = pyemu.LinearAnalysis(jco='freyberg_pp_reg_phimlim26.jcb')
plt.figure(figsize=(12,4))
ax = la.get_par_css_dataframe()['pest_css'].sort_values(ascending=False).plot(kind='bar')
ax.set_yscale('log')

### We can read in the covariance and look at correlation

In [ ]:
sc = pyemu.Schur('freyberg_pp_reg_phimlim26.jcb')
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df().head()

In [ ]:
R = covar.to_pearson()
plt.imshow(R.df(), interpolation='nearest', cmap='viridis')
plt.colorbar()

### Let's look at correlation
CSS suffers from the challenge that values with high CSS may be corelated with other parameters. We can check that out. Identifiability, on the other hand, tends to spread among the correlated parameters so that identifiability is suppressed from each of those parameters. This makes a big difference between what is "sensitive" vs. "identifiable"

In [ ]:
cpar = 'hkpp42'
R.df().loc[cpar][np.abs(R.df().loc[cpar])>.5]

## this is also good look at spatially

In [ ]:
def plot_identifiability_spatial(idp, nsingular, parlox, obslox,makelabels=False,figsize=(4,7)):
    ident_df = idp.la.get_identifiability_dataframe(nsingular)
    ident_df=ident_df[['ident']].join(parlox)    
    ident_df.drop('rch1',inplace=True)
    plt.figure(figsize=figsize)
    plt.plot(obslox.X, obslox.Y,'x')
    plt.scatter(ident_df.X, ident_df.Y, s=np.abs(ident_df.ident.values)*100,
                c=ident_df.ident.values, cmap='viridis')
    if makelabels:
        for cn, cg in pd.groupby(parlox, by='pp_num'):
            plt.gca().annotate(cn, xy=(cg.X,cg.Y), textcoords='data', fontsize=9)
    plt.axis('equal')
    plt.colorbar()
    plt.title('Identifiability with {0} singular values'.format(nsingular))
    plt.xlim(0,5000)
    plt.ylim(0,10000)
    plt.axis('off')

In [ ]:
plot_identifiability_spatial(idpar,12,parlox, obslox, makelabels=True)